# Data Aggregation and Group Operations
- Pandas provides a flexible groupby interface, enabling operations on *grpup*:
    - to slice
    - dice
    - summarize 

#### Esamples:
- average income by age
- average income by gender
- average income by role, gender and age
- average sell by city
- average sell by region and year
- sum and count sell by city
- min and max vote for student
- etc.

#### common operations:
- Split a pandas object into pieces using one or more keys (in the form of functions, arrays, or DataFrame column names)
- Calculate group summary statistics, like count, mean, or standard deviation, or a user-defined function
- Apply within-group transformations or other manipulations, like normalization, linear regression, rank, or subset selection
- Compute pivot tables
- Perform quantile analysis and other statistical group analyses

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## GroupBy Mechanics

## Group operations
- often referred to as: "split-apply-combine"

<img src="split-apply-combine.svg">

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


- Suppose we wanted to compute the mean of the data1 column using the labels from key1

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

- This grouped variable is now a GroupBy object
- It has not actually computed anything yet except for some intermediate data about the group key
- The idea is that this object has all of the information needed to then apply some operation to each of the groups

In [4]:
grouped.mean()

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

- we can even group by more than one attribute
- the resulting Series now has a **hierarchical index**:
    - consisting of the unique pairs of keys observed

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

### Pivoting a level of the index
- Pivot a level of the (necessarily hierarchical) index labels
    - from two-level index to: one-level index + columns
    - we have as many columns as the number of distinct values of the secondary index are
        - this is pivoting
#### Unstack
- Pivoting on the last element of the index is achievable through `unstack()`

In [16]:
means.unstack()

key2,one,two
key1,,
a,0.880536,0.478943
b,-0.519439,-0.555730


In [17]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


- we can pass arbitrary arrays to create the gorups
- the arrays must have the same lenght

In [13]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

In [19]:
# try pivoting (w/ unstack)
# df['data1'].groupby([states, years]).mean().unstack()

- not only series
- group by can be performed on the entire DataFrame

In [22]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [23]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


- notice that above we don't have a column for `key2`, even if it's not in the `gorup by`
    - `key2` is not numerical, so we cannot have a `mean` of it
    
But it can be a gouping key

In [24]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

- Regardless of the objective in using groupby, a generally useful GroupBy method is `size`
    - returns a Series containing group sizes

In [25]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups
- The GroupBy object supports iteration
- generates a sequence of 2-tuples containing:
    - the group key(s)
    - the chunk of data

In [26]:
for key, group in df.groupby('key1'):
    print('---\nThe key group is: {}\n---'.format(key))
    print(group)

---
The key group is: a
---
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
---
The key group is: b
---
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023


- with multiple key, remember to collect a tuple

In [27]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print('---------------------------------------\nThe keys of the group are: {}\n---------------------------------------'
          .format((k1, k2)))
    print(group)
    print()

---------------------------------------
The keys of the group are: ('a', 'one')
---------------------------------------
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435

---------------------------------------
The keys of the group are: ('a', 'two')
---------------------------------------
  key1 key2     data1     data2
1    a  two  0.478943  0.092908

---------------------------------------
The keys of the group are: ('b', 'one')
---------------------------------------
  key1 key2     data1     data2
2    b  one -0.519439  0.281746

---------------------------------------
The keys of the group are: ('b', 'two')
---------------------------------------
  key1 key2    data1     data2
3    b  two -0.55573  0.769023



### Piece of group by
- The easiest way to get just one group (one piece) is:
    1. to transform the group into a list
    2. create a dict from that list
    3. to select the key we are interested in

In [28]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


- indeed, the `grpupby()` 
    - returns an object on which we can perform aggregations
    - but it is not accessible as a dataframe (e.g., we cannot print it)

In [33]:
df.groupby('key1')

- we need to do the following instead

In [49]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023


### Selecting a Column or Subset of Columns
- Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation

`df.groupby('key1')['data1']`

`df.groupby('key1')[['data2']]`

is a *syntactic sugar* for:

`df['data1'].groupby(df['key1'])`

`df[['data2']].groupby(df['key1'])`

#### side note:
- **one single squared bracket** select a subset of the DataFrame as a **Series**
- **a double squared bracket** select a subset of the DataFrame as a **DataFrame**

In [95]:
print(type(df['data2']))
print(type(df[['data2']]))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


In [102]:
df.groupby(['key1', 'key2'])[['data2']].mean()
# df.groupby(['key1', 'key2'])['data2'].mean()

data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

In [103]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped.mean()

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

### Grouping with Dicts and Series

In [93]:
# Create a dataframe
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['Modena', 'Bologna', 'Ferrara', 'Mantova', 'Milano'],
                      index=['Sergio', 'Duccio', 'Renato', 'Corinna', 'Fabiana'])

# Add a few NA values
people.iloc[2:3, [1, 2]] = np.nan
people

,Modena,Bologna,Ferrara,Mantova,Milano
Sergio,1.047184,0.923948,-0.114150,0.405802,0.288452
Duccio,-0.434788,0.358756,-0.388245,2.128746,1.409605
Renato,-0.105434,NaN,NaN,-0.136972,-0.930489
Corinna,0.327497,1.303013,-1.409402,-0.144126,-0.716414
Fabiana,0.103614,-1.495719,-1.174894,2.613999,-0.689307


- we can use a dictionary to specify equivalences
    - if we want to create **custom groups**

In [94]:
mapping = {'Modena': 'Emilia Romagna', 
           'Bologna': 'Emilia Romagna', 
           'Ferrara': 'Emilia Romagna',
           'Mantova': 'Lombardia', 
           'Milano': 'Lombardia', 
           'Verona' : 'Veneto'}

In [95]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,Emilia Romagna,Lombardia
Sergio,1.856983,0.694254
Duccio,-0.464278,3.538351
Renato,-0.105434,-1.067461
Corinna,0.221108,-0.860540
Fabiana,-2.566998,1.924692


- the mapping is a dict:
    - thus, it can be a pandas object (a series) itsetf

In [96]:
map_series = pd.Series(mapping)
print(type(map_series))
map_series

<class 'pandas.core.series.Series'>


Modena     Emilia Romagna
Bologna    Emilia Romagna
Ferrara    Emilia Romagna
Mantova         Lombardia
Milano          Lombardia
Verona             Veneto
dtype: object

In [97]:
people.groupby(map_series, axis=1).count()

,Emilia Romagna,Lombardia
Sergio,3,2
Duccio,3,2
Renato,1,2
Corinna,3,2
Fabiana,3,2


### Grouping with Functions
- we can even employ the output of a function to perform grouping
    - e.g.: key with the same length
        - instead of creating a surrogate key with the output of `len(key)`
        - we can just pass the function `len`

In [98]:
people.groupby(len).sum()

,Modena,Bologna,Ferrara,Mantova,Milano
6,0.506962,1.282704,-0.502395,2.397577,0.767567
7,0.431111,-0.192705,-2.584296,2.469873,-1.405721


- try to group by the last char of the name

In [100]:
people.groupby(lambda x: x[-1]).sum()

,Modena,Bologna,Ferrara,Mantova,Milano
a,0.431111,-0.192705,-2.584296,2.469873,-1.405721
o,0.506962,1.282704,-0.502395,2.397577,0.767567


### Grouping by Index Levels
- we can have MultiIndex as well:
    - for istance, if we have 

In [124]:
columns = pd.MultiIndex.from_arrays([['Milano', 'Milano', 'Londra', 'Londra', 'Londra'],
                                    ['AC Milan', 'FC Internazionale', 'Arsenal','Chelsea','Tottenham']],
                                    names=['cty', 'team'])
hier_df = pd.DataFrame(np.random.randn(10, 5), columns=columns)
hier_df

cty     Milano                      Londra                    
team  AC Milan FC Internazionale   Arsenal   Chelsea Tottenham
0     0.941188         -0.931457 -0.124668  0.200696  0.180256
1    -0.320370         -1.596128 -1.281699  1.502586  0.653538
2    -0.319537          0.955094  0.261996  0.160793 -0.571681
3     0.351660          1.114980  1.183268  1.060941  0.510713
4    -0.938784         -0.546496  0.590030  1.482185  0.102118
5     0.265438          0.003193 -2.595012 -1.555569  1.102996
6     0.554737         -1.289012  0.385242 -1.717292 -1.018353
7     0.051635          0.503299 -0.543186 -0.506678  0.729653
8     0.434273         -1.133674  1.423953  0.266352 -0.854264
9    -0.550597         -0.619110  1.038933 -0.910611  0.529953

In [125]:
hier_df.groupby(level='cty', axis=1).count()

cty,Londra,Milano
0,3,2
1,3,2
2,3,2
3,3,2
4,3,2
5,3,2
6,3,2
7,3,2
8,3,2
9,3,2


## Data Aggregation
- We have seen already some aggregations, e.g., `min`, `max`, `sum`
- There are many others: on a group, after the `.` press "tab" and see the suggestions

In [ ]:
# df.groupby('key1').

- To quickly access to the doc, remember:

In [200]:
?df.groupby

In [201]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [202]:
grouped = df.groupby('key1')
grouped[['data1']].min()

,data1
key1,
a,-0.204708
b,-0.555730


In [203]:
grouped[['data1']].min().join(grouped[['data1']].max(), 
                              on='key1',
                              lsuffix='_min', 
                              rsuffix='_max')

,data1_min,data1_max
key1,,
a,-0.204708,1.965781
b,-0.555730,-0.519439


- get the number that corresponds to a particular quantile

In [204]:
grouped[['data1']].quantile(0.9)

,data1
key1,
a,1.668413
b,-0.523068


#### custom aggregation
- we can also design our own aggregation

In [205]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

/var/folders/c6/vq4_c9js51s48z2tp4sh0vvw0000gn/T/ipykernel_18235/663192242.py:3: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(peak_to_peak)


,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


In [206]:
# equivalnt to
grouped.agg(lambda x: x.max()-x.min())

/var/folders/c6/vq4_c9js51s48z2tp4sh0vvw0000gn/T/ipykernel_18235/1231709324.py:2: FutureWarning: ['key2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  grouped.agg(lambda x: x.max()-x.min())


,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


- we can get also description of the entire group

In [208]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.746672  1.109736 -0.204708  0.137118  0.478943  1.222362   
b      2.0 -0.537585  0.025662 -0.555730 -0.546657 -0.537585 -0.528512   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.965781   3.0  0.910916  0.712217  0.092908  0.669671  1.246435   
b    -0.519439   2.0  0.525384  0.344556  0.281746  0.403565  0.525384   

                          
           75%       max  
key1                      
a     1.319920  1.393406  
b     0.647203  0.769023

## Quantile and Bucket Analysis

In [308]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
frame

,data1,data2
0,-1.848329,1.919285
1,-0.707298,-1.596159
2,1.419703,-1.323971
3,-1.802546,-1.706602
4,-1.053745,0.015684
...,...,...
995,-0.658448,0.861923
996,0.032792,1.044042
997,-0.304001,0.565181
998,-0.231261,0.913172


### pd.cut
`cut` will choose the bins to be evenly spaced according to the values 

In [355]:
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-2.953, -1.425]
1    (-1.425, 0.0977]
2      (0.0977, 1.62]
3    (-2.953, -1.425]
4    (-1.425, 0.0977]
5    (-1.425, 0.0977]
6       (1.62, 3.143]
7       (1.62, 3.143]
8    (-2.953, -1.425]
9      (0.0977, 1.62]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.953, -1.425] < (-1.425, 0.0977] < (0.0977, 1.62] < (1.62, 3.143]]

In [356]:
type(quartiles[0])

pandas._libs.interval.Interval

- we extract the stats we are interested in form each group

In [358]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

In [359]:
grouped = frame.data1.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.953, -1.425]",-2.947150,-1.431365,87.0,-1.840108
"(-1.425, 0.0977]",-1.423202,0.095556,465.0,-0.565254
"(0.0977, 1.62]",0.102020,1.606996,386.0,0.755104
"(1.62, 3.143]",1.622775,3.142646,62.0,2.022503


### pd.qcut
`qcut` cuts the dataframe in quantiles:
- same number of element in each bean

In [360]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 4, labels=False)
grouped = frame.data1.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.947150,-0.770288,250.0,-1.326335
1,-0.769275,-0.046947,250.0,-0.372622
2,-0.045510,0.710961,250.0,0.342793
3,0.722538,3.142646,250.0,1.331895


## Example: Filling Missing Values with Group-Specific Values

In [366]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    2.254949
2         NaN
3    0.942245
4         NaN
5   -0.032192
dtype: float64

- we already saw the `fillna` function, which replace the `nan` with some value computed with a function

In [368]:
s.fillna(s.mean())

0    1.055001
1    2.254949
2    1.055001
3    0.942245
4    1.055001
5   -0.032192
dtype: float64

In [369]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.432115
New York      2.279705
Vermont      -0.745232
Florida       0.238593
Oregon       -0.650419
Nevada       -0.966286
California    0.026147
Idaho        -0.485245
dtype: float64

In [370]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.432115
New York      2.279705
Vermont            NaN
Florida       0.238593
Oregon       -0.650419
Nevada             NaN
California    0.026147
Idaho              NaN
dtype: float64

- we group on the fly, assigning the key from the `group_key`
 - remember: it's not in the dataframe, it's just a list that we use on-the-fly
 - i.e., make sure that the list is sorted as you want

In [373]:
data.groupby(group_key).mean()

East    0.695394
West   -0.312136
dtype: float64

In [374]:
data.groupby(group_key).apply(lambda group: group.fillna(group.mean()))

Ohio         -0.432115
New York      2.279705
Vermont       0.695394
Florida       0.238593
Oregon       -0.650419
Nevada       -0.312136
California    0.026147
Idaho        -0.312136
dtype: float64

- thi is equivalent to:
`data.groupby(group_key).apply(lambda x: x.fillna(x.mean()))`

- it' a simple labda function where we just use the `group` as name of the variable, insteat of the `x`

The function can employ also other resources to assign the value to each group
- for intance a `dict`

In [376]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -0.432115
New York      2.279705
Vermont       0.500000
Florida       0.238593
Oregon       -0.650419
Nevada       -1.000000
California    0.026147
Idaho        -1.000000
dtype: float64

In [392]:
close_px = pd.read_csv('stock_px_2.csv', parse_dates=True,
                       index_col=0)

In [393]:
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [394]:
close_px.tail()

,AAPL,MSFT,XOM,SPX
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


#### Pearson correlation coefficient
- https://en.wikipedia.org/wiki/Pearson_correlation_coefficient
    - gives us a measure of "how correlated are" two variables
![](pearson.png)

In [399]:
spx_corr = lambda x: x.corrwith(x['SPX'], method='pearson')

In [407]:
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


In [400]:
?df.pct_change

In [409]:
rets = close_px.pct_change().dropna()
rets.head()

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


- let's say that we want to grpup by year
- we shuold extract the year from the date

In [411]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)

In [413]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [415]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64